## ДЗ Домашнее задание по теме "Оценка точности модели, переобучение, регуляризация"
Дана статистика пользователей adult.csv.
Получите значения AUC для различных моделей и их параметров.

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import cross_val_score
from sklearn.metrics import roc_auc_score

In [2]:
data = pd.read_csv('adult.csv')
data.head()

,age,workclass,fnlwgt,education,educational-num,marital-status,occupation,relationship,race,gender,capital-gain,capital-loss,hours-per-week,native-country,income
0,25,Private,226802,11th,7,Never-married,Machine-op-inspct,Own-child,Black,Male,0,0,40,United-States,<=50K
1,38,Private,89814,HS-grad,9,Married-civ-spouse,Farming-fishing,Husband,White,Male,0,0,50,United-States,<=50K
2,28,Local-gov,336951,Assoc-acdm,12,Married-civ-spouse,Protective-serv,Husband,White,Male,0,0,40,United-States,>50K
3,44,Private,160323,Some-college,10,Married-civ-spouse,Machine-op-inspct,Husband,Black,Male,7688,0,40,United-States,>50K
4,18,?,103497,Some-college,10,Never-married,?,Own-child,White,Female,0,0,30,United-States,<=50K


In [3]:
#подготавливаю датафрейм

data.rename(columns={'educational-num': 'educational_num',
                     'marital-status': 'marital_status',
                     'capital-gain' : 'capital_gain',
                     'capital_loss': 'capital_loss',
                     'hours_per_week': 'hours_per_week',
                     'native-country': 'native_country'}, inplace=True)
data.head()

,age,workclass,fnlwgt,education,educational_num,marital_status,occupation,relationship,race,gender,capital_gain,capital-loss,hours-per-week,native_country,income
0,25,Private,226802,11th,7,Never-married,Machine-op-inspct,Own-child,Black,Male,0,0,40,United-States,<=50K
1,38,Private,89814,HS-grad,9,Married-civ-spouse,Farming-fishing,Husband,White,Male,0,0,50,United-States,<=50K
2,28,Local-gov,336951,Assoc-acdm,12,Married-civ-spouse,Protective-serv,Husband,White,Male,0,0,40,United-States,>50K
3,44,Private,160323,Some-college,10,Married-civ-spouse,Machine-op-inspct,Husband,Black,Male,7688,0,40,United-States,>50K
4,18,?,103497,Some-college,10,Never-married,?,Own-child,White,Female,0,0,30,United-States,<=50K


## Исследую данные

In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48842 entries, 0 to 48841
Data columns (total 15 columns):
age                48842 non-null int64
workclass          48842 non-null object
fnlwgt             48842 non-null int64
education          48842 non-null object
educational_num    48842 non-null int64
marital_status     48842 non-null object
occupation         48842 non-null object
relationship       48842 non-null object
race               48842 non-null object
gender             48842 non-null object
capital_gain       48842 non-null int64
capital-loss       48842 non-null int64
hours-per-week     48842 non-null int64
native_country     48842 non-null object
income             48842 non-null object
dtypes: int64(6), object(9)
memory usage: 5.6+ MB


In [5]:
data.describe()

,age,fnlwgt,educational_num,capital_gain,capital-loss,hours-per-week
count,48842.000000,4.884200e+04,48842.000000,48842.000000,48842.000000,48842.000000
mean,38.643585,1.896641e+05,10.078089,1079.067626,87.502314,40.422382
std,13.710510,1.056040e+05,2.570973,7452.019058,403.004552,12.391444
min,17.000000,1.228500e+04,1.000000,0.000000,0.000000,1.000000
25%,28.000000,1.175505e+05,9.000000,0.000000,0.000000,40.000000
50%,37.000000,1.781445e+05,10.000000,0.000000,0.000000,40.000000
75%,48.000000,2.376420e+05,12.000000,0.000000,0.000000,45.000000
max,90.000000,1.490400e+06,16.000000,99999.000000,4356.000000,99.000000


In [6]:
#оцениваю количество пропусков
iter_col = []

for i in data.columns:
    if data[i].dtype == 'object':
        iter_col.append(i)

b = len(data)
sum_percent = 0

for i in iter_col:
    percent = (len(data[data[i] == '?']) / b) * 100
    sum_percent += percent
    print(i, ':  ', len(data[data[i] == '?']), ' или  ', percent, '%')

print ()
print('Суммарно', sum_percent, '%')

workclass :   2799  или   5.7307235575938735 %
education :   0  или   0.0 %
marital_status :   0  или   0.0 %
occupation :   2809  или   5.751197739650301 %
relationship :   0  или   0.0 %
race :   0  или   0.0 %
gender :   0  или   0.0 %
native_country :   857  или   1.7546374022357807 %
income :   0  или   0.0 %

Суммарно 13.236558699479955 %


In [7]:
#определяю количство уникальных значений

print(len(data.workclass.unique()))
print(len(data.occupation.unique()))
print(len(data.native_country.unique()))

9
15
42


### Выводы

#### native_country
содержит 42 уникальных значения и 1.75% пропущенных данных, если я удалю эти строки, то не сильно повлияет на точность.

#### occupation
содержит 15 уникальных значения и 5.75% пропущенных данных, это довольно, так, что целесообразно закодирвоать их через labelEncoder().

#### workclass
ситуация аналогина occupation.

In [8]:
data.drop(np.where(data['native_country'] == '?')[0], inplace=True)

In [9]:
# заполняю медианами пропуски в workclass и occupation
from sklearn.preprocessing import LabelEncoder

#для workclass
le3 = LabelEncoder()
le3.fit(data['workclass'])

data.workclass.loc[data.workclass == '?'] = le3.inverse_transform(int(np.median(le3.transform(data['workclass']))))
                  

#для occupation
le8 = LabelEncoder()
le8.fit(data['occupation'])

data.occupation.loc[data.occupation == '?'] = le8.inverse_transform(int(np.median(le8.transform(data['occupation']))))

C:\Program Files\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
C:\Program Files\Anaconda3\lib\site-packages\pandas\core\indexing.py:189: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)
C:\Program Files\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [10]:
data.head()

,age,workclass,fnlwgt,education,educational_num,marital_status,occupation,relationship,race,gender,capital_gain,capital-loss,hours-per-week,native_country,income
0,25,Private,226802,11th,7,Never-married,Machine-op-inspct,Own-child,Black,Male,0,0,40,United-States,<=50K
1,38,Private,89814,HS-grad,9,Married-civ-spouse,Farming-fishing,Husband,White,Male,0,0,50,United-States,<=50K
2,28,Local-gov,336951,Assoc-acdm,12,Married-civ-spouse,Protective-serv,Husband,White,Male,0,0,40,United-States,>50K
3,44,Private,160323,Some-college,10,Married-civ-spouse,Machine-op-inspct,Husband,Black,Male,7688,0,40,United-States,>50K
4,18,Private,103497,Some-college,10,Never-married,Machine-op-inspct,Own-child,White,Female,0,0,30,United-States,<=50K


In [11]:
#кодирую категориальные данные

#workclass
le3 = LabelEncoder()
le3.fit(data['workclass'])
data['workclass'] = le3.transform(data['workclass'])

#education
le5 = LabelEncoder()
le5.fit(data['education'])
data['education'] = le5.transform(data['education'])

#marital_status
le7 = LabelEncoder()
le7.fit(data['marital_status'])
data['marital_status'] = le7.transform(data['marital_status'])

#marital_status
le8 = LabelEncoder()
le8.fit(data['occupation'])
data['occupation'] = le8.transform(data['occupation'])

#relationship
le9 = LabelEncoder()
le9.fit(data['relationship'])
data['relationship'] = le9.transform(data['relationship'])

#race
le10 = LabelEncoder()
le10.fit(data['race'])
data['race'] = le10.transform(data['race'])

#примерно на этом месте я понял, что всё это можно было бы упаковать в цикле в словарь,
    #но переделывать было лень

#gender
le11 = LabelEncoder()
le11.fit(data['gender'])
data['gender'] = le11.transform(data['gender'])

#native_country
le15 = LabelEncoder()
le15.fit(data['native_country'])
data['native_country'] = le15.transform(data['native_country'])

#income
le16 = LabelEncoder()
le16.fit(data['income'])
data['income'] = le16.transform(data['income'])

На этом, я счтаю, что данные подготовлены и приступаю к созданию моделей и оценке AUC.
Далее я использу KNeighborsClassifier и RandomForestClassifier

In [12]:
#разделяю датасет

from sklearn.model_selection import train_test_split

X = data.drop(['income'], axis = 1)
#X.head()

Y = data.income
#Y.head()

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size = 0.20)

#### KNeighborsClassifier

In [18]:
from sklearn.neighbors import KNeighborsClassifier

In [20]:
kNN = KNeighborsClassifier(n_neighbors = 3)
kNN.fit(X_train, y_train)
y_predict = kNN.predict(X_test)

In [21]:
print('Model Accuracy:', kNN.score(X_train, y_train))

Model Accuracy: 0.869959362300719


In [22]:
print ("AUC-ROC (test) = ", roc_auc_score(y_test, y_predict))

AUC-ROC (test) =  0.6250468492252567


In [23]:
cross_val_score(kNN, X_train, y_train, cv=2)

array([0.75836199, 0.76023757])

#### RandomForestClassifier

In [13]:
from sklearn.ensemble import RandomForestClassifier

C:\Program Files\Anaconda3\lib\site-packages\sklearn\ensemble\weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [14]:
RFCl = RandomForestClassifier(n_estimators = 100, criterion='gini', bootstrap=True, n_jobs=-1)
RFCl.fit(X_train, y_train)
y_predict = RFCl.predict(X_test)

In [15]:
print('Model Accuracy:', RFCl.score(X_train, y_train))

Model Accuracy: 0.9999218505783057


In [16]:
print ("AUC-ROC (test) = ", roc_auc_score(y_test, y_predict))

AUC-ROC (test) =  0.7734644936339398


In [17]:
cross_val_score(RFCl, X_train, y_train, cv=2)

array([0.85584037, 0.85531937])